In [117]:
import tensorflow as tf
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

In [118]:
def gm_to_generator(gm):
    for line_index in np.arange(len(gm)):
        if gm[line_index][0] == 'nb_cols':
            m = int(gm[line_index][3])
        elif gm[line_index][0] == 'nb_rows':
            n = int(gm[line_index][3])
        elif gm[line_index][0] == 'src_indx':
            src_index = np.array(gm[line_index+1], dtype=np.int32)
        elif gm[line_index][0] == 'par_indx':
            par_index = np.array(gm[line_index+1], dtype=np.int32)
        elif gm[line_index][0] == 'nzentries_per_row':
            start = line_index
            continue

    parity_matrix = np.zeros(shape=(n,m), dtype=np.float32)
    generator_line = 0

    for line_index in np.arange(start=start+1, stop=len(gm)):
        for a in gm[line_index]:
            parity_matrix[generator_line][int(a)] = 1
        generator_line += 1
    
    k = len(src_index)
    generator_matrix = np.zeros((m,k), dtype=np.float32)

    basis = np.zeros(k, dtype=np.float32)
    basis[0] = 1.

    for basis_index in np.arange(k):
        src_word = np.roll(basis, basis_index)
        generator_matrix[src_index, basis_index] = src_word
        
    for line_index in np.arange(n):
        res = np.matmul(parity_matrix[line_index,:], generator_matrix) % 2
        generator_matrix[par_index[line_index], :] = res

    generator_matrix = np.transpose(generator_matrix)

    return (generator_matrix, src_index, par_index)

In [119]:
main_alist_path = os.path.join('./','ml-decoding-gnbp_LA_reference')
alist_file_name = 'n64_k18_m46_dv2-4_dc3-4.alist'

alist_path = os.path.join(main_alist_path, alist_file_name)

In [120]:
with open(alist_path, "r") as myfile:
    data = myfile.readlines()
    a_list = [[int(x) for x in elt.replace(' \n','').replace(' ',',').split(',')] for elt in data]
    
n = int(a_list[0][0])
k = int(n - a_list[0][1])
n_k = int(a_list[0][1])

print(f'n: {n}')
print(f'k: {k}')
print(f'n-k: {n_k}')

n: 64
k: 18
n-k: 46


In [121]:
var_idx = a_list[4:4+n]
check_idx = a_list[4+n:4+n+1+n_k]

parity_check_matrix = np.zeros([n,n_k])

for i in range(n):
    for idx in var_idx[i]:
        if idx != 0:
            parity_check_matrix[i][(idx-1)] = 1

H_non_systematic = parity_check_matrix.T

In [122]:
from maths import GaussJordanGF2, LegacyGaussJordanGF2

gj = GaussJordanGF2(n=n, k=k, output_all=True, trainable=False)
# gj = LegacyGaussJordanGF2(n=n, k=k, output_all=True, trainable=False)

G_non_systematic, G_systematic, H_systematic, source_index, parity_index = gj(H_non_systematic)

In [123]:
syndrom_systematic = tf.reduce_sum(tf.matmul(G_systematic, H_systematic, transpose_b=True) % 2)
syndrom_non_systematic = tf.reduce_sum(tf.matmul(G_non_systematic, H_non_systematic, transpose_b=True) % 2)

print(f'Syndrom systematic = {syndrom_systematic}')
print(f'Syndrom non systematic = {syndrom_non_systematic}')

Syndrom systematic = 0.0
Syndrom non systematic = 0.0


In [124]:
from encoders import LinearBlockEncoder

data = tf.eye(num_rows=k, dtype=tf.float32)

encoder = LinearBlockEncoder(n=n, k=k, trainable=False)
encoder.w_generator_matrix = G_systematic

codeword = encoder(data)
source_data = tf.gather(codeword, indices=tf.range(k, dtype=tf.int32), axis=1)
print(f'Source data errors (systematic): {tf.reduce_sum(tf.abs(source_data - data))}')

encoder.w_generator_matrix = G_non_systematic

codeword = encoder(data)
source_data = tf.gather(codeword, indices=source_index, axis=1)
print(f'Source data errors (non systematic): {tf.reduce_sum(tf.abs(source_data - data))}')

print(f'Differences between G_non_systematic and G_systematic = {tf.reduce_sum(tf.abs(G_non_systematic - G_systematic))}')

Source data errors (systematic): 0.0
Source data errors (non systematic): 0.0
Differences between G_non_systematic and G_systematic = 0.0


In [125]:
main_gm_path = os.path.join('./','ml-decoding-gnbp_LA_reference')
gm_file_name = 'n64_k18_m46_dv2-4_dc3-4.gm'

gm_path = os.path.join(main_gm_path, gm_file_name)

with open(gm_path, "r") as myfile:
    data = myfile.readlines()
    gm = [[x for x in elt.replace(' \n','').replace('\n','').replace(' ',',').split(',')] for elt in data]

generator_matrix, src_index, par_index = gm_to_generator(gm)

print(f'G_non_systematic errors = {tf.reduce_sum(tf.abs(G_non_systematic - generator_matrix))}')
print(f'G_systematic errors = {tf.reduce_sum(tf.abs(G_systematic - generator_matrix))}')
print(f'Source index errors = {tf.reduce_sum(tf.abs(source_index - src_index))}')
print(f'Parity index errors = {tf.reduce_sum(tf.abs(parity_index - par_index))}')

syndrom_systematic = tf.reduce_sum(tf.matmul(generator_matrix, H_systematic, transpose_b=True) % 2)
syndrom_non_systematic = tf.reduce_sum(tf.matmul(generator_matrix, H_non_systematic, transpose_b=True) % 2)

print(f'Syndrom systematic = {syndrom_systematic}')
print(f'Syndrom non systematic = {syndrom_non_systematic}')

G_non_systematic errors = 0.0
G_systematic errors = 0.0
Source index errors = 0
Parity index errors = 722
Syndrom systematic = 0.0
Syndrom non systematic = 0.0


In [126]:
main_save_path = os.path.join('./', 'reference')
npz_file_name = 'n64_k18_m46_dv2-4_dc3-4.npz'

npz_path = os.path.join(main_save_path, npz_file_name)
np.savez(npz_path, G_non_systematic=G_non_systematic, G_systematic=G_systematic, H_systematic=H_systematic, H_non_systematic=H_non_systematic, src_index=source_index)

In [127]:
main_gm_path = os.path.join('./','ml-decoding-gnbp_LA_reference')
gm_file_name = 'n64_k46_m18_dv2-3_dc9-10.gm'

gm_path = os.path.join(main_gm_path, gm_file_name)

with open(gm_path, "r") as myfile:
    data = myfile.readlines()
    gm = [[x for x in elt.replace(' \n','').replace('\n','').replace(' ',',').split(',')] for elt in data]

generator_matrix, src_index, par_index = gm_to_generator(gm)

print(f'G_non_systematic errors = {tf.reduce_sum(tf.abs(G_non_systematic - generator_matrix))}')
print(f'G_systematic errors = {tf.reduce_sum(tf.abs(G_systematic - generator_matrix))}')
print(f'Source index errors = {tf.reduce_sum(tf.abs(source_index - src_index))}')
print(f'Parity index errors = {tf.reduce_sum(tf.abs(parity_index - par_index))}')

syndrom_systematic = tf.reduce_sum(tf.matmul(generator_matrix, H_systematic, transpose_b=True) % 2)
syndrom_non_systematic = tf.reduce_sum(tf.matmul(generator_matrix, H_non_systematic, transpose_b=True) % 2)

print(f'Syndrom systematic = {syndrom_systematic}')
print(f'Syndrom non systematic = {syndrom_non_systematic}')

InvalidArgumentError: Incompatible shapes: [18,64] vs. [46,64] [Op:Sub]

In [ ]:
# code_matrices = np.load(f'reference/BCH_{n}_{k}.npz')ml-decoding-gnbp_LA_reference
code_matrices = np.load(f'ml-decoding-gnbp_LA_reference/n64_k46_m18_dv3_dc10-11.npz')

try:
    reference_systematic_generator_matrix = tf.convert_to_tensor(code_matrices['G'], dtype=tf.float32)
    reference_non_systematic_generator_matrix = tf.convert_to_tensor(code_matrices['G'], dtype=tf.float32)
except:
    reference_systematic_generator_matrix = tf.convert_to_tensor(code_matrices['G_systematic'], dtype=tf.float32)
    reference_non_systematic_generator_matrix = tf.convert_to_tensor(code_matrices['G_non_systematic'], dtype=tf.float32)
reference_systematic_paritycheck_matrix = tf.convert_to_tensor(code_matrices['H_systematic'], dtype=tf.float32)
reference_nonsystematic_paritycheck_matrix = tf.convert_to_tensor(code_matrices['H_non_systematic'], dtype=tf.float32)

print(f'H_non_systematic errors = {tf.reduce_sum(tf.abs(H_non_systematic - reference_nonsystematic_paritycheck_matrix))}')
print(f'H_systematic errors = {tf.reduce_sum(tf.abs(H_systematic - reference_systematic_paritycheck_matrix))}')
print(f'G_non_systematic errors = {tf.reduce_sum(tf.abs(G_non_systematic - reference_non_systematic_generator_matrix))}')
print(f'G_systematic errors = {tf.reduce_sum(tf.abs(G_systematic - reference_systematic_generator_matrix))}')
print(f'Source index: {source_index}')
print(f'Parity index: {parity_index}')

H_non_systematic errors = 311.0
H_systematic errors = 400.0
G_non_systematic errors = 868.0
G_systematic errors = 400.0
Source index: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45]
Parity index: [46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
